# 0414 CNN 嘗試部分
為了與上次的NN做比較<br>
在相同都是使用mnist手寫數據集的情況下<br>
將Learning Rate(0.01)、batch_size(100)、epochs(20)都調整成一致，<br>
以觀察樣本內(train data)跟樣本外(test data)的正確率<br>

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow.keras.datasets import fashion_mnist

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [4]:
x_train.shape

(60000, 28, 28)

## Channel
CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

(28,28) --> (28, 28, 1)

In [5]:
x_train = x_train.reshape(60000,28,28,1) /255

In [6]:
x_test = x_test.reshape(10000,28,28,1) /255

In [7]:
x_train[87].shape

(28, 28, 1)

In [8]:
y_train[87]

7

In [9]:
from tensorflow.keras.utils import to_categorical

In [10]:
y_train = to_categorical(y_train)

In [11]:
y_test = to_categorical(y_test)

## 讀入必要的函式

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D     #因為flitter是3*3矩陣 所以是2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

## Step 1 打造函數學習機 (CNN) 

In [13]:
model = Sequential()

In [14]:
model.add(Conv2D(16,(3,3), padding='same',  #16個fliter
               input_shape=(28,28,1),       #輸入的資料(1 channel) *第一層需要打而已
               activation='relu'))

In [15]:
# 輸出16個 28*28 的矩陣
# 事實上是 (28, 28, 16)

In [16]:
model.add(MaxPooling2D(pool_size=(2,2)))  #每 2*2 格中選最大的

In [17]:
# 輸出變成(14, 14, 16)

In [18]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [19]:
# 輸出變成(14 ,14, 32)   
# 原本輸入是 14*14 且 16個Channel 每一個新的filter會對16個有不同的加權
# =>共3*3*16個parameter+1個activation function
# 會形成一張 14*14 的計分表
# 最後因為有32個filter，所以總共會輸出有32張計分表->變成32Channel再去下一層

In [20]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [21]:
# Output (7, 7, 32)

In [22]:
model.add(Conv2D(64, (3,3), padding='same',      #num of parameter: (3*3*32+1)*64
                activation='relu'))

In [23]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [24]:
model.add(Flatten())  #為了將3*3的圖拉平放進標準機器學習

In [25]:
model.add(Dense(54, activation='relu'))

In [26]:
model.add(Dense(10, activation='softmax'))

## 看一下我們的神經網路

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

In [28]:
model.compile(loss='mse', optimizer=SGD(lr=0.01),   #因為是用gradient decent 所以需要lr rate來決定一次要移動多少幅度
            metrics=['accuracy'])            #希望能邊訓練邊說明多少正確率

## Step 2. fit

In [29]:
model.fit(x_train, y_train, batch_size=100, epochs=20)

Train on 60000 samples
Epoch 1/20
60000/60000 [==============================] - 42s 700us/sample - loss: 0.0899 - accuracy: 0.1064
Epoch 2/20
60000/60000 [==============================] - 40s 662us/sample - loss: 0.0896 - accuracy: 0.1172
Epoch 3/20
60000/60000 [==============================] - 47s 790us/sample - loss: 0.0893 - accuracy: 0.1400- loss: 0.0893 - accuracy: 0.
Epoch 4/20
60000/60000 [==============================] - 48s 799us/sample - loss: 0.0890 - accuracy: 0.1664
Epoch 5/20
60000/60000 [==============================] - 39s 655us/sample - loss: 0.0887 - accuracy: 0.2018
Epoch 6/20
60000/60000 [==============================] - 40s 672us/sample - loss: 0.0884 - accuracy: 0.2477
Epoch 7/20
60000/60000 [==============================] - 42s 702us/sample - loss: 0.0879 - accuracy: 0.2725
Epoch 8/20
60000/60000 [==============================] - 42s 700us/sample - loss: 0.0873 - accuracy: 0.2908
Epoch 9/20
60000/60000 [==============================] - 37s 617us/sample -

## Step 3. 預測

In [30]:
result = model.predict_classes(x_test)

In [31]:
def my_predict(n):
    print('CNN預測是:',result[n])
    X = x_test[n].reshape(28, 28)
    plt.imshow(X, cmap='Greys')

In [32]:
from ipywidgets import interact_manual

In [33]:
interact_manual(my_predict, n=(0, 99999))

interactive(children=(IntSlider(value=49999, description='n', max=99999), Button(description='Run Interact', s…

<function __main__.my_predict(n)>

In [34]:
score = model.evaluate(x_test, y_test)

10000/10000 [==============================] - 2s 232us/sample - loss: 0.0446 - accuracy: 0.6984


In [35]:
loss, acc = score

In [36]:
print('測試資料的正確率為', acc)

測試資料的正確率為 0.6984


## 把我們的Model存起來

In [37]:
model.save('machine_learning_Mnist_CNN.h5')  #h5為特殊檔案

# 比較與NN的正確率差別

In [40]:
NN_traindata_accuracy = 0.6840
NN_testdata_accuracy = 0.6845
print('NN樣本內正確率:', NN_traindata_accuracy)
print('NN樣本外正確率:', NN_testdata_accuracy)
print('CNN樣本內正確率:0.7025')
print('CNN樣本外正確率:', acc)
print('NN比CNN的正確率差不多，應該可以多嘗試修改其他的參數!')

NN樣本內正確率: 0.684
NN樣本外正確率: 0.6845
CNN樣本內正確率:0.7025
CNN樣本外正確率: 0.6984
NN比CNN的正確率差不多，應該可以多嘗試修改其他的參數!
